# Text Classification From Scratch
In this exercise we will practice working with Python by implementing a text classification algorithm. This will serve as an example that introduces you informally to Machine Learning. Most of the concepts that you will see later in this course can be understood by relating them to this example. 

Before solving this exercise we recommend you 

   1. <a href='https://www.continuum.io/downloads' target="_blank">Install Python, Numpy and MatplotLib through Anaconda by clicking here!</a>
   2. Try to solve the [theory exercises](theory_mathquiz.ipynb).  
   

## 1. The Problem
In this exercise we will try to help the Danish Business Authority (DBA). Each company in Denmark is registered at DBA with a <i>purpose statement</i> and an <i>industry code</i>. For example:  
<table>
 <tr>
  <td><b>Purpose Statement</b></td>
  <td><b>Industry Code</b></td>
 </tr>
<tr>
  <td>Make sushi to people</td>
  <td>1113</td>
</tr>
<tr>
  <td>Make spaghetti to people</td>
  <td>1113</td>
</tr>
<tr>
  <td>Fix peoples laptop</td>
  <td>324</td>
</tr>
<tr>
  <td>Make lasagna to people</td>
  <td>1113</td>
</tr>
<tr>
  <td>Fix peoples smartphones</td>
  <td>324</td>
</tr>
</table>

In the above example is seems that the industry code 1113 has something to do with making food for people, it might be the industry code for restaurants. Similarly 324 might be the company code for technology related business. The above example is made up. You will later be working with real data.  

The DBA suspect some companies provide incorrect industry codes. They want to detect industry codes that are probably incorrect and help new companies select the right industry code. To help with this we want an algorithm that

    Given a string (the purpose statement of a company) outputs the most likely associated industry code. 

If we are succesful in doing so, we would be able to 
    1. suggest industry codes to new companies 
    2. find industry codes that are not the most likely (they might be errors) 
In the real world there are <b>1000</b>'s different industry codes. To make the exercise easier, we will only consider two different industry codes: *Restaurants* and *Computer Programming*.
<!--The raw data will contain information for many different industry codes, buall the industries, but we'll filter it later-->

## 2. The Learning Approach
We shall try to solve the above problem by <i>learning</i>. To do so we have a several thousand examples of purpose statements and their corresponding industry codes. We then need to <i>learn</i> from the examples how to compute an industry code from a purpose statement (we call the examples our *data set*). 

You can think of learning how to do this as learning how to measure which group of purpose statements the given purpose statement is mostly similar to. And this we need to capture in some function we can invoke when required.

There are many many ways to do this! In this exercise the the approach is as follows: we assume companies in different industries use different words when they write the purpose statement for the company. If a company has the industry code for *Restaurant* the purpose statements is more likely to contain words like <i>food</i>, <i>eating</i> and <i>wine</i>. Similarly, companies with the industry code for *Computer Programming* are more likely to contain words like <i>applications</i>, <i>coding</i> and <i>development</i>. 

For each word we estimate a *probability* of corresponding to the industry code of *Restuarant* or *Computer Programming*. Hopefully, this will allow us to infer *the most likely industry* code given a purpose statement.  

## 3. Let's look at the data
The following Python code prints out some of the data. Unfortunately the purpose statements are in Danish, however, understanding the purpose statements are not that important. The reason that we look at the data is to 
    1. notice real world data is noisy and has errors
    2. investigate if companies in different industries really do use different words. 

Don't worry too much about the code, it roughly prints the following three things:
    1. The first 10 examples
    2. The 20 most frequent purpose statements of *Restaurants*. 
    3. The 20 most frequent purpose statements of *Computer Programming*.

In [1]:
#Ensure plots are inlines
%matplotlib inline
# Import NumPy and MatplotLib pyplot
import numpy as np 
import matplotlib.pyplot as plt

import pandas as pd # Dataframe library
from IPython.display import display      

import os

import urllib

# Download the data if it does not exist. 
filename = 'branchekoder_formal.gzip'
if not os.path.exists(filename):
    #os.system('wget https://users-cs.au.dk/jallan/ml/data/{0}'.format(filename))
    
    with open(filename,'wb') as fh:
        fh.write(urllib.request.urlopen("https://users-cs.au.dk/jallan/ml/data/%s" % filename).read())
    
    
data = pd.read_csv(filename, compression='gzip')
# Let's see the first 10 data points. There is much more data here than we use in the exercise.
print('\nThe first 10 examples:')
display(data.head(10))
print('-'*80)
# show which strings that occur the most in the data

rest = data[data.branchekode==561010] # notice the cool filtering going on here.
rest_vc = rest.formal.value_counts().to_frame()
print("\n\nThe 20 most frequent purpose statements of Restaurants:")
display(rest_vc[0:20])
print('-'*80)

cpu = data[data.branchekode==620100]
cpu_vc = cpu.formal.value_counts().to_frame() 
print("\n\nThe 20 most frequent purpose statements of Computer Programming")
display(cpu_vc[0:20])


The first 10 examples:


,Unnamed: 0,enhedsnummer,branchekode,formal
0,0,2551095,642020,Selskabets formål er at eje aktier og anparter...
1,1,4000405664,642020,Selskabets formål er at eje aktier i danske ak...
2,2,4001251528,642020,"Selskabets formål er at drive handel-, industr..."
3,3,4001248440,642020,Selskabets formål er at eje aktier i Un Maskin...
4,4,3937295,682040,Selskabets formål er at eje og handle med fast...
5,5,4001248443,999999,Selskabets formål er formueforvaltning
6,6,3412826,494200,"Selskabets formål er at drive handel, herunder..."
7,7,4001248449,433200,Selskabets formål er at drive erhvervsmæssig v...
8,8,4402812,682040,Selskabets formål er passiv formueadministration
9,9,4001248455,702200,Selskabets formål er at fungere som holdingsel...


--------------------------------------------------------------------------------


The 20 most frequent purpose statements of Restaurants:


,formal
Selskabets formål er at drive restaurationsvirksomhed.,134
Selskabets formål er at drive restaurationsvirksomhed,60
Selskabets formål er at drive restaurationsvirksomhed og dermed beslægtet virksomhed.,52
Selskabets formål er at drive restaurationsvirksomhed samt enhver i forbindelse hermed stående virksomhed.,27
Selskabets formål er at drive restaurationsvirksomhed og dermed beslægtet virksomhed,26
Selskabets formål er at drive restaurationsvirksomhed samt dermed beslægtet virksomhed.,26
Selskabets formål er at drive restaurationsvirksomhed og hermed beslægtet virksomhed.,24
Selskabets formål er at drive restaurationsvirksomhed samt hermed beslægtet virksomhed.,21
"Selskabets formål er at drive restaurationsvirksomhed, samt enhver i forbindelse hermed stående virksomhed.",19
Selskabets formål er at drive restauration og dermed beslægtet virksomhed.,18


--------------------------------------------------------------------------------


The 20 most frequent purpose statements of Computer Programming


,formal
Selskabets formål er at udøve virksomhed med handel og service samt aktiviteter i tilknytning hertil.,29
"Selskabets formål er at drive virksomhed med handel og service, samt enhver i forbindelse hermed stående virksomhed.",14
Selskabets formål er at drive handel og industri.,12
Selskabets formål er salg og udvikling af internetrelaterede ydelser samt dermed beslægtet virksomhed.,10
"Selskabets formål er at drive IT-konsulentvirksomhed, samt hermed beslægtet virksomhed.",9
Selskabets formål er at drive konsulentvirksomhed.,7
Selskabets formål er at drive virksomhed med softwareudvikling samt enhver i forbindelse hermed stående virksomhed.,7
"Selskabets formål er at drive informationsteknologisk virksomhed og anden i forbindelse hermed stående virksomhed, herunder at besidde andele i andre selskaber.",6
Selskabets formål er at drive IT-virksomhed.,5
"Selskabets formål er at drive virksomhed med software udvikling, samt enhver i forbindelse hermed stående virksomhed.",5


## 4. Your task
You have to do the following two things: 

  1. Understand the mathematical model
  2. Implement the classification algorithm
  
The mathematical model will be explained in section 5. If you are less mathematically inclined this might be a bit difficult to understand. Do not despair. Give it a good attempt, spend 20-30 minutes trying, carefully reading each sentence. Then progress to section 6, which gives a detailed explanation of how to code the classification algorithm. It should be possible to follow without having a perfect understanding of section 5. 

When done coding, we recommend you re-read section 5 to understanding what is going on. 

<!-- You have to do the following five tasks. Each will be explained in more detail below.
  1. <b>Preprocess</b>: Clean the purpose statements (strings) data and make each string to list of words;
  2. <b>Preprocess</b>: Make sentences to bag of words vectors of numbers (easier to work with numbers than strings);
  3. <b>Learn Probabilities</b>: Compute class probabilites for each word. 
  I.e. what is the probability of seeing the word "beer" in a purpose statement from a company with industry code "Computer Programming" (and for "Restaurant");
  4. <b>Learn Class Probabilities</b>: Compute class probabilities, if we pick a random company but do not see the word list how likely is each class;
  5. <b>Predict</b>: Make an algorihm that given a purpose statement outputs the most likely associated industry code.

The final step is of course very interesting and we should consider it before we do all the work.
First however, you should download the exercise code and data to a separate folder so you can work with it from your favourite editor and ipython. 
You are welcome to copy it into a notebook like this (your own copy) if you prefer.
The files already contains a lot of code and tell you where you need to bring your code and finish it. 
Remember to read the comments for each function before you complete them.

LINK TO CODE THEY NEED TO DOWNLOAD!

!! Read understanding the mathematical model first, then try to impllement, then read the math again!! -->

## 5. Understanding the mathematical model
To work with probabilities we need a probabilistic model. This section will describe and explain the model we will use. To do this we use probability theory concepts like *conditional independence* and *Bayes theorem*. To remind you about these concepts we have attached several links to Wikipedia you might find useful.   

Given a string $s$ (purpose statement) we first preprocess $s$ into a list of words $w_1,\dots, w_m$. We then want to compute the probability of each class given the words $w_1,\dots,w_m$. Notice that this is the <a href="https://en.wikipedia.org/wiki/Conditional_probability" target="_blank">conditional probability</a> 

$$\Pr(c\;|\;w_1,...,w_m)$$

Here $c$ denotes the class. In our case we have that $c\in\{'\text{Computer Programming}','\text{Restaurants}'\}$. The most likely class is then given by  

$$
\textrm{arg max}_c \;\Pr(c\mid w_1,\dots, w_m)
$$

If you are not familiar with <a href="https://en.wikipedia.org/wiki/Arg_max#Definition" target="_blank">$\textrm{arg max}$</a> you should look it up on <a href="https://en.wikipedia.org/wiki/Arg_max#Definition" target="_blank">Wikipedia</a>. In our case, the above equation means the argument $c$ that maximizes $\Pr(c\mid w_1,\dots,w_m)$. But this requires us to compute $\Pr(c\mid w_1,\dots,w_m)$. By <a href="https://en.wikipedia.org/wiki/Bayes%27_theorem#Statement_of_theorem" target="_blank">Bayes theorem</a> this is given

$$
\Pr(c\mid w_1,\dots,w_m) = \frac{\Pr(w_1,\dots,w_m \mid c) \Pr(c)}{\Pr(w_1,\dots,w_m)}
$$

Remember that we only care about $\Pr(c\mid w_1,\dots,w_m)$ such that we can compute the $\text{arg max}_c$ above. This means we only care about which $c$ causes the maximal value and not what the maximal value is. But the denominator $\Pr(w_1,\dots,w_m)$ is the same for all classes $c$, therefore we can consider it a constant that we can ignore. In other words, it is sufficient to compute $\Pr(w_1,\dots,w_m\mid c)$ and $\Pr(c)$.

It is easy to estimate $\Pr(c)\approx \frac{\text{# purpose statements in class } c}{\text{total # purpose statements}}$.

To compute $\Pr(w_1,\dots, w_m\mid c)$ we make the following assumption:<br>

<div style="border: 1px solid #333; padding: 16px; margin: 16px; "><b>Assumption:</b> <br>
Given a class $c$ the words $w_1,\dots,w_m$ are independent. This is called <a href="https://en.wikipedia.org/wiki/Conditional_independence#Formal_definition" target="_blank">conditional independence</a>: 
$$
\Pr(w_1,\dots, w_m |c) = \prod_{i=1}^m \Pr(w_i |c)
$$
In other words, the probabilites of seeing each word $w_i$ are independent given $c$. </div>

It is important to notice this may or may not be a reasonable assumption, but we are going to use it here. 

This concludes the description of our model. The following two subsections provide a summary of the *mathematics* and the resulting *algorithm*. 

### 5. 1. A brief summary of the math

\begin{align}
\text{arg max}_c \Pr[c \mid w_1,\dots w_m] &=
\text{arg max}_c \frac{\Pr(w_1,\dots,w_m \mid c) \Pr(c)}{\Pr(w_1,\dots,w_m)}\\
&=\text{arg max}_c \Pr(w_1,\dots,w_m \mid c) \Pr(c)\\
&=\text{arg max}_c  \prod_{i=1}^m \Pr(w_i\mid c) \;\;\frac{\text{# purpose statements in class } c}{\text{total # purpose statements}} 
\end{align}

Try to read the above and see if you understand what happens at each equality. It is okay if you do not understand all the steps. Write down which steps are bugging you, and ask your TA!  

### 5. 2. Summary of algorithm
Our algorithm should take as input a purpose statement as a string $s$. First the string $s$ is made into a list of words $W=w_1,\dots,w_m$. The algorithm then computes 

$$\Pr('\text{computer programming}' \mid W) \quad\quad\text{ and }\quad\quad \Pr('\text{restaurants}' \mid W)$$ 

using the formula above. This algorithm is called <a href="https://en.wikipedia.org/wiki/Naive_Bayes_classifier" target="_blank">Naive Bayes</a>. If you are interested <a href="https://en.wikipedia.org/wiki/Naive_Bayes_classifier" target="_blank">Wikipedia</a> has a bunch of information on this algorithm. When your curiosity has been satisfied follow the instructions below to implement this algorithm so we can apply it.

## 6. What you need to code
We have provided you with some [starter code](code_text_classification.py). 
Each of the following subsections will describe the functionality you need to add. If you succeed you will have a working Naive Bayes classifier. 

To help you, the code will have comments '### YOUR CODE' and '### END CODE' to indicate where you need to add code. 

The following list should give a overview of the things you need to implement. 

<ol>
 <li><b>Preprocess</b>
  <ol>
    <li>Cleaning the purpose statement strings</li>
    <li>Bag of words vector</li>
  </ol>
 </li>
 <li><b>Learn</b>: Compute word and class probabilities.</li>
 <li><b>Predict</b>: Implement the prediction algorithm. </li>
 <li><b>Evaluate</b>: How good does our algorithm perform?</li>
</ol>

<!--
  1. <b>Preprocess</b>: Clean the purpose statements (strings) data and make each string to list of words;
  2. <b>Preprocess</b>: Make sentences to bag of words vectors of numbers (easier to work with numbers than strings);
  3. <b>Learn Probabilities</b>: Compute class probabilites for each word. 
  I.e. what is the probability of seeing the word "beer" in a purpose statement from a company with industry code "Computer Programming" (and for "Restaurant");
  4. <b>Learn Class Probabilities</b>: Compute class probabilities, if we pick a random company but do not see the word list how likely is each class;
  5. <b>Predict</b>: Make an algorihm that given a purpose statement outputs the most likely associated industry code. -->

### 6. 1. a) (Preprocess) Cleaning the purpose statement strings
You need to complete the implementation of **standardize_strings**, **split_strings** and **remove_irrelevant_words**.
For that, list comprehensions may come very much in handy.
You can test your implementations by running

    python code_text_classification.py -test 
   

### 6. 1. b) (Preprocess) Bag of words vectors
In machine learning we prefer to work with vectors of numbers instead of strings. The goal of this subsection is to make each purpose statement string into a vector. First we find *all* the unique words in *all* the purpose statement strings. Call them $V = \{w_1,\dots, w_{|V|}\}$, where $V$ stands for vocabulary.

We choose to represent a sentence $s$ (a list of words) as a vector of length |V|, where V[i] is equal to the number of times $w_i$ is in the sentence $s$. This means we choose to forget the ordering of words and we will get very sparse vectors.

<div style='border: 1px solid #333; padding: 16px; margin: 16px;'><b>Example</b>: Given strings <br>

$$S=\{'\text{hello world}', '\text{foo bar baz}', '\text{hello foo}', '\text{bar baz world}'\}$$<br>

The vocabulary is then the list of all unique words <br>

$$V=\{'\text{hello}', '\text{world}', '\text{foo}', '\text{bar}', '\text{baz}'\}$$ <br>

The sentence $s=['\text{hello}', '\text{world}', '\text{hello}, '\text{world}', '\text{foo}', '\text{foo}', '\text{foo}']$ is then represented by the vector $v=[2, 2, 3, 0, 0]$. Notice that the ordering of the words are lost and many sentences will be represented by very sparse vectors.  </div>


Complete the implementation of **make_vocabulary_and_index** and **words_to_vectors**.
Test your implementation with 

    python code_text_classification.py -test

### 6.2. ("Learn") Compute Word and Class Probabilities
We now want to compute $\Pr(c)$ and $\Pr(w\mid c)$. Remember that $\Pr(c)$ is the probability that a random purpose statement belongs to the class $c$. This can be estimated as 

$$\Pr(c) = \frac{\textrm{# purpose statements in class }c}{\textrm{total # purpose statements}}$$

The word class probabilites $\Pr(w\mid c)$ can be estimated in a similar way. For each class, count the number of occurences of each word and divide by the number of words. 

$$\Pr(w\mid c) = \frac{1+\textrm{# occurences of word } w \textrm{ in class c}}{\textrm{# words in class c} + |V|}$$<br>

The '+1' is added to fix a problem. Some words might not occur in any class. Adding the '+1' ensures the probability won't be 0 which would've caused problems. The $|V|$ in the denominator is to ensure that the probabilites sum to one.

Complete the function, **compute_class_word_probabilities** and test with 

    python code_text_classification.py -test


### 6. 3. (Predict) Implement the prediction algorithm
We have implemented the training algorithm for you. It basically calls the methods you have implemented to compute the probabilities. You must now use use these to compute 

\begin{align}
\text{arg max}_c \Pr[c \mid w_1,\dots w_m] =...=\text{arg max}_c  \prod_{i=1}^m \Pr(w_i\mid c) \Pr(c)
\end{align}

to predict the most likely classes for a given input of purpose statement strings. 

Implement the **predict** method.

Please note that you must <b>ignore</b> words not seen before / in the training data. 

### 6. 4. (Evaluate) How good does our algorithm perform? 
Run 

    python code_text_classification.py -run 
    
and see how well you did. 

This will test your implementation on approximately a thousand data points not seen by the training algorithm and test how well your algorithm generalize to new data. We get around 95 percent correct (accuracy) on the unseen data (may fluctuate due to randomness). The code will also visualize some statistics and show you some of the errors the model makes.

### 6.5 What mistakes does it make
When you run the algorithm it will print purpose statements that it misclassifies. Can you come up with explanations for why some of these fails.




